### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [15]:
from sqp import *

TESTSTRING = "02002102000101000000102002110000021102201100000121101100102200102212002011112010100202020220021011110001020002102020211001102210020111001100000102100022100110201210022100020000101002210202100021000220220211202211110002221011010211000211202201021002102200012201101110222110002022012210202020020102100202211110202001122020000110020222220022110010020002102120002010010000211002021102102121210202221122000110202101020002020022200021000211020211022210200121022200010211002201101110220220110202110202210020212102102120002210002202112110210020001010002002000202102121222022121022201210211202020022100222101102112100221202021001010211020210102110202211200202000000000022102020000021111220012110201121010002002020000120200222022110202011002101002110010002120221100011000002100220222202021110222102200022001101011122021021111120021100010210222100222110202210102002221000021202020210200201101001120002211121011000212002000122022200121011120000210111011111020112221002002202"
# TESTSTRING = TESTSTRING[:40]
numpy.printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [16]:
def run_sqp(test_string, lmbd):
    global pbm
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l), np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    # x0 = np.random.rand(n**2)
    x0 = np.hstack((x0, np.dot(C, x0))).copy()
    global res
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-10,'disp': 2, "maxiter":5000},
            bounds=bounds, 
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [24]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.05)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)
print("Eqn con rhs: ")
print(pbm.EqnCon(res.x))
print("Ineq con rhs: ")
print(pbm.IneqCon(res.x))

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 2.0369771899802482
            Iterations: 995
            Function evaluations: 10863
            Gradient evaluations: 991
The original MLE estimated transition matrix is: 


array([[0.27645051, 0.38566553, 0.33788396],
       [0.42298289, 0.40342298, 0.17359413],
       [0.15444015, 0.5019305 , 0.34362934]])

The New Transiiton Matrix after the SQP is: 


array([[0.66081637, 0.18765745, 0.1517356 ],
       [0.16275349, 0.15049787, 0.68705674],
       [0.69730625, 0.15252575, 0.15048624]])

Eqn con rhs: 
[0.00020942 0.00030809 0.00031824]
Ineq con rhs: 
[ 3.95577150e-06  5.69808812e-06  7.03908222e-06  7.42074491e-06
  2.62391695e-03  3.64851405e-03  6.90939681e-06  7.70031371e-06
  3.95627336e-06  1.16462983e-03  2.36566343e-04  4.99437565e-02
  5.09683506e-02  2.96399457e-06  3.75563802e-06  1.10796994e-03
  2.22964549e-06  5.35376836e-02  5.45622777e-02  8.52675752e-05
  1.99588680e-06  1.66083906e-06  5.24372361e-02  5.34618301e-02
 -1.24756874e-07  6.66988434e-07  5.36631794e-02  5.46877734e-02
  2.05024026e-04  4.83978084e-07  1.02459899e-03  6.78149449e-06
  7.57238992e-06  6.42403822e-06  7.21490027e-06  7.92992295e-07
  4.73198474e-02  5.09137745e-02  4.98133269e-02  5.10392702e-02
 -1.20155580e-07 -4.74573568e-07  5.08359707e-02  5.10407132e-02
  3.59614102e-03  3.65502605e-03  3.95252420e-03  3.82778337e-06
  3.47029426e-06  3.51613363e-03  3.72087688e-03  6.18140840e-06
  1.26002749e-04  5.57008012e-06  5.21263874e-06  6.25246001e-06
  1.26932375e-04  1.227222

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [18]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.03)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 11.873691483450873
            Iterations: 208
            Function evaluations: 2177
            Gradient evaluations: 204
The original MLE estimated transition matrix is: 


array([[0.27645051, 0.38566553, 0.33788396],
       [0.42298289, 0.40342298, 0.17359413],
       [0.15444015, 0.5019305 , 0.34362934]])

The New Transiiton Matrix after the SQP is: 


array([[9.99998398e-01, 8.24135002e-07, 8.18209425e-07],
       [8.13212794e-07, 8.05473666e-07, 9.99998411e-01],
       [9.99998463e-01, 7.74218848e-07, 8.31986000e-07]])

In [19]:
res

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 11.873691483450873
       x: [ 1.000e+00  8.241e-07 ...  3.000e-02  6.987e-09]
     nit: 208
     jac: [-1.186e+01 -7.009e-06 ...  1.000e+00  1.000e+00]
    nfev: 2177
    njev: 204